In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_ENDPOINT"] = os.getenv("LANGCHAIN_ENDPOINT")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

In [ ]:
loader=WebBaseLoader("https://docs.smith.langchain.com/tutorials/Administrators/manage_spend")
loader

In [ ]:
docs=loader.load()
docs

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [ ]:
documents

In [ ]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [ ]:
from langchain_community.vectorstores import FAISS
vectorstoredb = FAISS.from_documents(documents, embeddings)

In [ ]:
vectorstoredb

In [ ]:
query = "LangSmith has two usage limits: total traces and extended"
result = vectorstoredb.similarity_search(query)
result[0].page_content

In [ ]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
"""
Answer the following question based only on the provided context:
<context>
{context}
</context>
"""
)

document_chain=create_stuff_documents_chain(llm, prompt)
document_chain

In [ ]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "LangSmith has two usage limits: total traces and extended",
    "context": [Document(page_content="LangSmith has two usage limits: total traces and extended retention traces. These correspond to the two metrics we've been tracking on our usage graph. We can use these in tandem to have granular control over spend.")]
})

In [ ]:
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrival_chain=create_retrieval_chain(retriever, document_chain)

In [ ]:
retrival_chain

In [ ]:
response=retrival_chain.invoke({"input": "LangSmith has two usage limits: total traces and extended"})
response["answer"]

In [ ]:
response

In [ ]:
response['context']